In [8]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from bgreference import refseq 
import collections
from collections import Counter
from itertools import islice
import numpy as np

def get_mut(x):    
    try:
        #triplet = '{}_{}'.format(refseq('sarscov2', '0', x['POS']-1, 3).replace('\n', 'N'), x['ALT'])
        triplet = '{}{}{}_{}'.format(refseq('sarscov2', '0', x['pos']-1, 1).replace('\n', 'N'), x['ref'], refseq('sarscov2', '0', x['pos']+1, 1).replace('\n', 'N'), x['alt'])
    except:
        triplet = 'NNN'
    return triplet

def order_muts_full():    
    order = []
    first = ['A', 'C', 'G', 'T']
    pyr = ['C', 'T']
    for p in pyr:
        for mut in first:
            if mut != p:
                for f in first:
                    for f2 in first:
                        comb = '{}{}{}_{}'.format(f, p, f2, mut)
                        order.append(comb)
    first = ['T', 'G', 'C', 'A']
    pyr = ['G', 'A']
    for p in pyr:
        for mut in first:
            if mut != p:
                for f in first:
                    for f2 in first:
                        comb = '{}{}{}_{}'.format(f, p, f2, mut)
                        order.append(comb)    
    return order

def order_muts96():

    order = []
    first = ['A', 'C', 'G', 'T']
    pyr = ['C', 'T']
    for p in pyr:
        for mut in first:
            if mut != p:
                for f in first:
                    for f2 in first:
                        comb = '{}{}{}_{}'.format(f, p, f2, mut)
                        order.append(comb)

    return order

def slicing_window(seq, n=3):
    """
    Fast slicing window
    :param seq:
    :param n:
    :return:
    """    
    
    it = iter(seq)
    result = ''.join(islice(it, n))    
    
    if len(result) == n:
        yield result    
        
    for elem in it:
        result = result[1:] + elem
        #print(result)
        yield result
        
l = refseq('sarscov2', '0', 1, 100000000).rstrip()
count_genome_compo = Counter(list(slicing_window(l)))
print(count_genome_compo)

Counter({'TTT': 1004, 'AAA': 923, 'TTA': 876, 'TGT': 858, 'TTG': 817, 'ACA': 809, 'ATT': 773, 'AAT': 761, 'CTT': 738, 'ATG': 725, 'TAA': 719, 'CAA': 703, 'GTT': 700, 'ACT': 674, 'TGA': 630, 'TAT': 622, 'AAC': 615, 'TAC': 609, 'AGA': 605, 'AAG': 580, 'CTA': 561, 'TGG': 554, 'GTG': 552, 'TCA': 549, 'TGC': 547, 'TCT': 542, 'GAA': 535, 'GCT': 521, 'TTC': 518, 'AGT': 507, 'CTG': 495, 'CAT': 484, 'ATA': 471, 'GTA': 469, 'CAC': 459, 'GGT': 454, 'GAT': 440, 'CAG': 438, 'TAG': 427, 'ACC': 376, 'GCA': 372, 'CCA': 354, 'CCT': 344, 'GAC': 340, 'ATC': 339, 'AGG': 329, 'AGC': 301, 'GAG': 297, 'CTC': 287, 'GGA': 282, 'GTC': 269, 'GGC': 223, 'TCC': 209, 'GCC': 187, 'CGT': 171, 'ACG': 164, 'GGG': 134, 'CCC': 116, 'TCG': 113, 'CGC': 97, 'CGA': 95, 'GCG': 88, 'CGG': 76, 'CCG': 74})


In [13]:
df_muts = pd.read_csv('../output/paired/allmuta_annot_paired.csv', sep ='\t')
df_muts

,mutation,branch,ref,alt,pos,csqn_type,AF,SAMPLE
0,A81T,NaN,A,T,81,upstream_gene_variant,0.019211,ERR4365562.dup.realg.ann
1,A170G,NaN,A,G,170,upstream_gene_variant,0.018787,ERR4365562.dup.realg.ann
2,C241T,NaN,C,T,241,upstream_gene_variant,0.998438,ERR4365562.dup.realg.ann
3,A462G,NaN,A,G,462,missense_variant,0.013709,ERR4365562.dup.realg.ann
4,A462T,NaN,A,T,462,missense_variant,0.028860,ERR4365562.dup.realg.ann
...,...,...,...,...,...,...,...,...
224066,C66A,NaN,C,A,66,upstream_gene_variant,0.020772,ERR4306728.dup.realg.ann
224067,C21T,NaN,C,T,21,upstream_gene_variant,1.000000,ERR4242079.dup.realg.ann
224068,C21T,NaN,C,T,21,upstream_gene_variant,0.500000,ERR4366956.dup.realg.ann
224069,C44T,NaN,C,T,44,upstream_gene_variant,0.040816,ERR4206654.dup.realg.ann


In [14]:
letters = ['A', 'T', 'C', 'G']
#remove indels
df_unique_muts = df_muts[(df_muts['ref'].isin(letters))&(df_muts['alt'].isin(letters))]
#filter 0.01 to 0.5
df_unique_muts = df_unique_muts[(df_unique_muts['AF']>0.01) & (df_unique_muts['AF']<0.5)]
df_unique_muts = df_unique_muts[['pos', 'ref', 'alt', 'SAMPLE']]
#print(df_unique_muts)
dfcounts = df_unique_muts.groupby(["SAMPLE"]).size().to_frame('count').reset_index()
print(dfcounts.sort_values(by=['count']))
print("TOTAL samples: " + str(len(dfcounts)))
#Filter by number of mutations?
dfcounts = dfcounts[dfcounts['count']>20]
print("TOTAL samples with more than 20 mutations: " + str(len(dfcounts)))
df_unique_muts = df_unique_muts[df_unique_muts['SAMPLE'].isin(dfcounts['SAMPLE'])]
print(dfcounts['count'].sum())


                        SAMPLE  count
1201  ERR4206361.dup.realg.ann      1
8225  ERR4332999.dup.realg.ann      1
6584  ERR4307278.dup.realg.ann      1
1508  ERR4207159.dup.realg.ann      1
6583  ERR4307277.dup.realg.ann      1
...                        ...    ...
7886  ERR4309130.dup.realg.ann    227
5725  ERR4306061.dup.realg.ann    229
8925  ERR4365519.dup.realg.ann    239
5110  ERR4305152.dup.realg.ann    265
7733  ERR4308926.dup.realg.ann    275

[9465 rows x 2 columns]
TOTAL samples: 9465
TOTAL samples with more than 20 mutations: 1357
68603


In [15]:
df_unique_muts['FULL_MUT'] = df_unique_muts.apply(get_mut, axis = 1)
df_unique_muts_count=df_unique_muts[['FULL_MUT', 'SAMPLE','pos']]
df_unique_muts_count = df_unique_muts_count.groupby(['FULL_MUT','SAMPLE']).count().reset_index()
#192 channels
#list_order = order_muts_full()
#96 channels
list_order = order_muts96()
print (list_order)
d = {}
#build dataframe. Rows are triplets, columns samples.
for ix,mut in enumerate(list_order):
    d[mut]=[]
    for sample in list(dfcounts['SAMPLE']):
        if len(df_unique_muts_count[(df_unique_muts_count["FULL_MUT"]==mut)&(df_unique_muts_count["SAMPLE"]==sample)].pos.values)>0:
            d[mut].append(df_unique_muts_count[(df_unique_muts_count["FULL_MUT"]==mut)&(df_unique_muts_count["SAMPLE"]==sample)].pos.values[0])
        else:
            d[mut].append(0)

#print(list(dfcounts['SAMPLE']))
df=pd.DataFrame.from_dict(d,orient='index')
df.columns = list(dfcounts['SAMPLE'])
print(df)
df.to_csv("sigProfiler_snvs96_20muta.tsv",sep="\t",index=False)

['ACA_A', 'ACC_A', 'ACG_A', 'ACT_A', 'CCA_A', 'CCC_A', 'CCG_A', 'CCT_A', 'GCA_A', 'GCC_A', 'GCG_A', 'GCT_A', 'TCA_A', 'TCC_A', 'TCG_A', 'TCT_A', 'ACA_G', 'ACC_G', 'ACG_G', 'ACT_G', 'CCA_G', 'CCC_G', 'CCG_G', 'CCT_G', 'GCA_G', 'GCC_G', 'GCG_G', 'GCT_G', 'TCA_G', 'TCC_G', 'TCG_G', 'TCT_G', 'ACA_T', 'ACC_T', 'ACG_T', 'ACT_T', 'CCA_T', 'CCC_T', 'CCG_T', 'CCT_T', 'GCA_T', 'GCC_T', 'GCG_T', 'GCT_T', 'TCA_T', 'TCC_T', 'TCG_T', 'TCT_T', 'ATA_A', 'ATC_A', 'ATG_A', 'ATT_A', 'CTA_A', 'CTC_A', 'CTG_A', 'CTT_A', 'GTA_A', 'GTC_A', 'GTG_A', 'GTT_A', 'TTA_A', 'TTC_A', 'TTG_A', 'TTT_A', 'ATA_C', 'ATC_C', 'ATG_C', 'ATT_C', 'CTA_C', 'CTC_C', 'CTG_C', 'CTT_C', 'GTA_C', 'GTC_C', 'GTG_C', 'GTT_C', 'TTA_C', 'TTC_C', 'TTG_C', 'TTT_C', 'ATA_G', 'ATC_G', 'ATG_G', 'ATT_G', 'CTA_G', 'CTC_G', 'CTG_G', 'CTT_G', 'GTA_G', 'GTC_G', 'GTG_G', 'GTT_G', 'TTA_G', 'TTC_G', 'TTG_G', 'TTT_G']
       ERR4082720.dup.realg.ann  ERR4082755.dup.realg.ann  \
ACA_A                         0                         0   
ACC_A        